In [ ]:
import os
import re
from dotenv import load_dotenv
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Load environment variables
load_dotenv()


# Watsonx chat function
def watsonx_llm(messages):
    """
    Generate a chat completion using IBM Watsonx models.

    Parameters:
    - messages (list of dict): The messages for the chat session.
    - model_size (int, optional): Specifies the model size. Default is 8.

    Returns:
    - str: The generated response content.
    """
    model_id = f"meta-llama/llama-3-1-8b-instruct"

    # Fetch environment variables
    api_key = os.getenv("WX_KEY")
    url = os.getenv("WX_URL")
    project_id = os.getenv("WX_PID")

    if not all([api_key, project_id, model_id]):
        raise Exception(
            "Missing required environment variables: WX_KEY, WX_URL, or WX_PID."
        )

    # Adjusted parameters
    params = {
        GenParams.DECODING_METHOD: "sample",
        GenParams.MAX_NEW_TOKENS: 256,
        GenParams.TEMPERATURE: 0.0,
        GenParams.STOP_SEQUENCES: ["<|eot_id|>", "<|eom_id|>"],
    }

    # Setup credentials
    credentials = Credentials(api_key=api_key, url=url)

    # Instantiate ModelInference
    model_inference = ModelInference(
        model_id=model_id, params=params, credentials=credentials, project_id=project_id
    )

    # Call the chat method
    response = model_inference.chat(messages=messages)

    # Extract and return the generated content
    return response["choices"][0]["message"]["content"]


# Define the ReAct Agent class
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return watsonx_llm(self.messages)


# Define available actions
def calculate(what):
    try:
        return eval(what)
    except Exception as e:
        return f"Error in calculation: {e}"


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weighs 50 lbs"


def calculator(expression):
    """
    A more advanced calculator that supports complex mathematical expressions.
    """
    try:
        # Safeguard to limit the operations
        result = eval(expression, {"__builtins__": {}}, {})
        return result
    except Exception as e:
        return f"Error: {e}"


# Add tools to known actions
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
    "calculator": calculator,
}

# Initialize the prompt
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

average_dog_weight:
e.g. average_dog_weight: Collie
Returns the average weight of a dog when given the breed.

calculator:
e.g. calculator: (5 + 10) * (2 ** 3)
Runs a more advanced calculation and returns the result. It can handle complex mathematical expressions.

Example session:

Question: What is the result of (3 + 7) * 2?
Thought: I should use the calculator tool to compute the result.
Action: calculator: (3 + 7) * 2
PAUSE

You will be called again with this:

Observation: The result is 20

You then output:

Answer: The result of (3 + 7) * 2 is 20.
""".strip()

# Define the query function with ReAct loop
action_re = re.compile(r"^Action: (\w+): (.*)$")


def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return


# Example usage
question = "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
query(question)

In [ ]:
import os
import re
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# Load environment variables
load_dotenv()


# Initialize Watsonx LLM
def watsonx_llm(messages):
    model_id = "meta-llama/llama-3-1-8b-instruct"
    api_key = os.getenv("WX_KEY")
    url = os.getenv("WX_URL")
    project_id = os.getenv("WX_PID")

    if not all([api_key, project_id, model_id]):
        raise Exception(
            "Missing required environment variables: WX_KEY, WX_URL, or WX_PID."
        )

    params = {
        GenParams.DECODING_METHOD: "sample",
        GenParams.MAX_NEW_TOKENS: 256,
        GenParams.TEMPERATURE: 0.0,
        GenParams.STOP_SEQUENCES: ["<|eot_id|>", "<|eom_id|>"],
    }

    credentials = Credentials(api_key=api_key, url=url)
    model_inference = ModelInference(
        model_id=model_id, params=params, credentials=credentials, project_id=project_id
    )
    response = model_inference.chat(messages=messages)
    return response["choices"][0]["message"]["content"]


# Preprocess Documents for RAG
loader = DirectoryLoader(
    "/Users/charan/VSCode/EMEA/Comarch_Telco_OSS_Assistant/data",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)
split_docs = text_splitter.split_documents(docs)

# Embedding and Vector Store
embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url=os.getenv("WX_URL"),
    apikey=os.getenv("WX_KEY"),
    project_id=os.getenv("WX_PID"),
)
vectorstore = Chroma.from_documents(
    split_docs, collection_name="agentic-rag-chroma", embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})


# Define Retrieval Tool
def get_context(question):
    results = retriever.invoke(question)
    return "\n".join([doc.page_content for doc in results])


# Define available actions
known_actions = {
    "get_context": get_context,
}


# Define ReAct Agent
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return watsonx_llm(self.messages)


# Initialize the prompt
prompt = """
You operate as a retrieval-augmented assistant with tools to fetch information from documents.
You follow this loop:

1. Thought: Describe your reasoning process.
2. Action: Run a tool to get additional context or answer the question.
3. Observation: Report the results of the action.
4. Answer: Use the retrieved information and your reasoning to answer.

Available actions:

get_context:
e.g., get_context: What is the definition of Generic Network Slice Template?
Fetches context about the given question from stored documents.
""".strip()

# Define the query function with ReAct loop
action_re = re.compile(r"^Action: (\w+): (.*)$")


def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result


# Example Usage
question = "What is Generic Network Slice Template as defined by GSMA?"
query(question)

In [ ]:
# Example Usage
question = "What is Generic Network Slice Template as defined by GSMA?"
query(question)